In [6]:
import numpy as np
import pandas as pd
import os

In [7]:


def merge_csv_files(input_folder, output_file):
    # List all CSV files in the folder
    csv_files = [f for f in os.listdir(input_folder) if f.endswith('.csv')]
    
    if not csv_files:
        print("No CSV files found in the folder.")
        return
    
    # Read and merge all CSV files
    df_list = []
    for file in csv_files:
        file_path = os.path.join(input_folder, file)
        print(f"Reading {file_path} ...")
        df = pd.read_csv(file_path)
        df_list.append(df)
    
    merged_df = pd.concat(df_list, ignore_index=True)
    
    # Save to a single CSV
    merged_df.to_csv(output_file, index=False)
    print(f"Merged {len(csv_files)} files into {output_file}")

if __name__ == "__main__":
    input_folder = "../data/raw"  # change this to your folder path
    output_file = "../data/interim/merged.csv"
    merge_csv_files(input_folder, output_file)


Reading ../data/raw\jalgaon 2010-01-01 to 2011-12-31.csv ...
Reading ../data/raw\jalgaon 2012-01-01 to 2013-12-31.csv ...
Reading ../data/raw\jalgaon 2014-01-01 to 2016-08-31.csv ...
Reading ../data/raw\jalgaon 2016-09-01 to 2018-12-31.csv ...
Reading ../data/raw\jalgaon 2019-01-01 to 2021-08-31.csv ...
Reading ../data/raw\jalgaon 2021-09-01 to 2022-12-31.csv ...
Reading ../data/raw\jalgaon 2023-01-01 to 2025-08-11.csv ...
Merged 7 files into ../data/interim/merged.csv


In [8]:
df = pd.read_csv("../data/interim/merged.csv")


In [9]:
df.head

<bound method NDFrame.head of          name    datetime  tempmax  tempmin  temp  feelslikemax  feelslikemin  \
0     jalgaon  2010-01-01     26.4     14.5  20.7          26.4          14.5   
1     jalgaon  2010-01-02     30.1     16.0  22.3          29.0          16.0   
2     jalgaon  2010-01-03     30.7     17.9  23.6          29.9          17.9   
3     jalgaon  2010-01-04     29.0     18.3  23.1          29.0          18.3   
4     jalgaon  2010-01-05     27.1     14.4  20.4          27.0          14.4   
...       ...         ...      ...      ...   ...           ...           ...   
5697  jalgaon  2025-08-07     34.3     25.1  29.2          36.7          25.1   
5698  jalgaon  2025-08-08     32.6     26.0  29.8          36.7          26.0   
5699  jalgaon  2025-08-09     34.8     27.8  31.1          39.8          30.4   
5700  jalgaon  2025-08-10     35.3     27.2  31.1          38.3          29.5   
5701  jalgaon  2025-08-11     31.2     26.7  28.7           0.0           0.0  

In [10]:
df.columns

Index(['name', 'datetime', 'tempmax', 'tempmin', 'temp', 'feelslikemax',
       'feelslikemin', 'feelslike', 'dew', 'humidity', 'precip', 'precipprob',
       'precipcover', 'preciptype', 'snow', 'snowdepth', 'windgust',
       'windspeed', 'winddir', 'sealevelpressure', 'cloudcover', 'visibility',
       'solarradiation', 'solarenergy', 'uvindex', 'severerisk', 'sunrise',
       'sunset', 'moonphase', 'conditions', 'description', 'icon', 'stations'],
      dtype='object')

# Weather Data Columns – Explanation & Cleaning Guide  
*(Based on Visual Crossing Weather Documentation)*
 https://www.visualcrossing.com/resources/documentation/weather-data/weather-data-documentation/

---

## 1. Temperature Columns

### **temp**
**Meaning:**  
The average temperature of the day in °C.

**Cleaning:**  
- Check for impossible values (< –90 or > 60).  
- Fill missing values using forward/backward fill.  
- Plot distribution and remove extreme outliers using IQR.

---

### **tempmin**
**Meaning:**  
Minimum temperature of the day.

**Cleaning:**  
- Must always be ≤ temp and tempmax.  
- Fix inconsistencies like *tempmin > temp*.  
- Replace missing values using interpolation.

---

### **tempmax**
**Meaning:**  
Maximum temperature recorded.

**Cleaning:**  
- Must always be ≥ temp and tempmin.  
- Remove unrealistic spikes using rolling median smoothing.

---

## 2. Humidity & Dew Columns

### **humidity**
**Meaning:**  
Relative humidity percentage (0–100%).

**Cleaning:**  
- Clamp values within 0–100.  
- If multiple consecutive missing values, interpolate using spline.  
- Identify unrealistic patterns (e.g., constant values for many days).

---

### **dew**
**Meaning:**  
Dew point temperature in °C.

**Cleaning:**  
- dew should not exceed tempmax.  
- Handle missing values using temperature–humidity–based estimation if needed.

---

## 3. Pressure Columns

### **sealevelpressure**
**Meaning:**  
Atmospheric pressure adjusted to sea level (hPa).

**Cleaning:**  
- Valid range: **950–1050 hPa** for most regions.  
- Remove sudden unrealistic jumps.  
- Impute missing values using seasonal trend decomposition.

---

## 4. Wind Variables

### **windspeed**
**Meaning:**  
Average wind speed (km/h).

**Cleaning:**  
- Replace unrealistic values > 150 km/h unless storm data.  
- Ensure no negative values.  
- Use log-transform for highly skewed data.

---

### **windgust**
**Meaning:**  
Maximum instantaneous wind speed.

**Cleaning:**  
- windgust must always be ≥ windspeed.  
- Remove outliers using percentile clipping.  
- Fill missing gust values with *windspeed + 20%*.

---

### **winddir**
**Meaning:**  
Wind direction in degrees (0–360).

**Cleaning:**  
- Normalize direction using modulo: `winddir % 360`.  
- Replace missing values with previous valid direction.

---

## 5. Cloud Cover

### **cloudcover**
**Meaning:**  
Cloud cover percentage (0–100%).

**Cleaning:**  
- Fix invalid values above 100 or below 0.  
- Use moving average to smooth sudden noise.  
- Handle long missing blocks using random forest imputation.

---

## 6. Visibility

### **visibility**
**Meaning:**  
Horizontal visibility distance (km).

**Cleaning:**  
- Valid range: **0–20 km** (depending on source).  
- Replace 0 values only if not due to fog conditions.  
- Impute missing values using median grouped by month.


In [11]:
df_copy = df[['datetime','tempmax','tempmin',
              'temp','dew','humidity','precip',
              'precipprob', 'precipcover', 'preciptype',
               'windgust','windspeed', 'winddir',
               'sealevelpressure', 'cloudcover', 'visibility',]]

In [12]:
df_copy.shape

(5702, 16)

In [13]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5702 entries, 0 to 5701
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   datetime          5702 non-null   object 
 1   tempmax           5702 non-null   float64
 2   tempmin           5702 non-null   float64
 3   temp              5702 non-null   float64
 4   dew               5702 non-null   float64
 5   humidity          5702 non-null   float64
 6   precip            5702 non-null   float64
 7   precipprob        5702 non-null   float64
 8   precipcover       5702 non-null   float64
 9   preciptype        2220 non-null   object 
 10  windgust          4260 non-null   float64
 11  windspeed         5702 non-null   float64
 12  winddir           5702 non-null   float64
 13  sealevelpressure  5702 non-null   float64
 14  cloudcover        5702 non-null   float64
 15  visibility        3500 non-null   float64
dtypes: float64(14), object(2)
memory usage: 71

In [14]:
df_copy.isnull().sum()

datetime               0
tempmax                0
tempmin                0
temp                   0
dew                    0
humidity               0
precip                 0
precipprob             0
precipcover            0
preciptype          3482
windgust            1442
windspeed              0
winddir                0
sealevelpressure       0
cloudcover             0
visibility          2202
dtype: int64

In [15]:
df_copy['preciptype'].unique()

array([nan, 'rain'], dtype=object)

In [16]:
df_copy['preciptype'] = df_copy['preciptype'].map({'rain': 1}).fillna(0).astype(int)


C:\Users\Care\AppData\Local\Temp\ipykernel_11792\1654495325.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy['preciptype'] = df_copy['preciptype'].map({'rain': 1}).fillna(0).astype(int)


In [17]:
print(df_copy['preciptype'].unique())
print(df_copy['preciptype'].isnull().sum())

[0 1]
0


In [18]:
df_copy.isnull().sum()

datetime               0
tempmax                0
tempmin                0
temp                   0
dew                    0
humidity               0
precip                 0
precipprob             0
precipcover            0
preciptype             0
windgust            1442
windspeed              0
winddir                0
sealevelpressure       0
cloudcover             0
visibility          2202
dtype: int64

In [19]:
df_copy['windgust'].unique()

array([ nan, 19.1, 22. , 18.7, 20.5, 26.6, 24.5, 20.9, 22.7, 18.4, 16.2,
       14. , 14.8, 33.1, 29.9, 14.4, 24.1, 23.4, 27.4, 23. , 17.3, 28.8,
       23.8, 19.8, 38.9, 20.2, 31. , 34.2, 25.2, 28.1, 24.8, 27.7, 25.6,
       15.8, 21.2, 25.9, 18. , 30.2, 30.6, 19.4, 32.4, 39.2, 36.7, 32.8,
       34.6, 31.3, 35.3, 31.7, 39.6, 29.5, 16.6, 34.9, 22.3, 21.6, 26.3,
       38.2, 27. , 28.4, 36.4, 36. , 32. , 41.4, 33.5, 40. , 37.8, 37.4,
       40.3, 42.8, 48.6, 41. , 49. , 54. , 52.2, 54.4, 47.2, 42.1, 45.4,
       44.6, 46.4, 46.1, 40.7, 41.8, 51.1, 49.7, 53.3, 51.5, 70.6, 43.2,
       33.8, 47.9, 46.8, 60.1, 35.6, 42.5, 43.9, 37.1, 29.2, 16.9, 15.5,
       15.1, 17.6, 48.2, 44.3, 45. , 51.8, 45.7, 60.5, 49.3, 58.3, 47.5,
       43.6, 52.6, 58.7, 61.9, 71.6, 50.8, 38.5, 53.6, 52.9, 55.1, 50. ,
       50.4, 55.8, 13.3, 57.6, 54.7, 59. , 56.5, 64.1, 56.2, 62.6, 58. ,
       55.4, 13.7, 11.9, 13. , 12.6, 57.2, 61.2, 60.8])

In [20]:
df_copy['windgust'] = df_copy['windgust'].ffill().bfill()


C:\Users\Care\AppData\Local\Temp\ipykernel_11792\1202939143.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy['windgust'] = df_copy['windgust'].ffill().bfill()


In [21]:
df_copy.isnull().sum()

datetime               0
tempmax                0
tempmin                0
temp                   0
dew                    0
humidity               0
precip                 0
precipprob             0
precipcover            0
preciptype             0
windgust               0
windspeed              0
winddir                0
sealevelpressure       0
cloudcover             0
visibility          2202
dtype: int64

In [22]:
df_copy['visibility'].unique()

array([   nan,    4. ,    3. ,    7. ,   10. ,    2.5,   11.5,   12. ,
         20. ,   15. ,   32. ,   21.2,   22.4,   23.3,   21.6,   22.9,
         21.8,   22.3,   22.7,   23.1,   20.6,   20.5,   22.2,   18.8,
         20.9,   24.6,   21.5,   21.7,   20.4,   22.5,   20.1,   19.1,
         22. ,   12.3,   19.9,   19.6,   16.2,   13.7,   16.4,   18.1,
         11.3,   20.2,   15.4,   11.2,   19.2,   21.3,   20.7,   18.6,
         17.8,   12.5,   14.4,   19.7,   16.9,   17. ,   22.1,   21.4,
         21.9,   17.4,   20.8,   17.1,   18.2,   17.2,   15.3,   18.3,
         17.9,   18.9,   23.5,   22.8,   23. ,   21.1,   21. ,   15.6,
         11. ,   14.8,   22.6,   19.3,   10.2,   16.7,   18.5,   20.3,
         19. ,   14.9,    8.1,   18.7,   16.8,   17.5,   24.1,   23.9,
         19.8,   13.3,   18.4,   16. ,   14.7,   15.8,   10.7,   15.7,
         23.2,   17.6,   16.5,   10.3,   15.9, 1513.2,   11.8,   19.4,
         14.6,   23.8,   23.7,   23.6])

In [23]:
df_copy['visibility'] = df_copy['visibility'].ffill().bfill()


C:\Users\Care\AppData\Local\Temp\ipykernel_11792\597574824.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy['visibility'] = df_copy['visibility'].ffill().bfill()


In [24]:
df_copy.isnull().sum()

datetime            0
tempmax             0
tempmin             0
temp                0
dew                 0
humidity            0
precip              0
precipprob          0
precipcover         0
preciptype          0
windgust            0
windspeed           0
winddir             0
sealevelpressure    0
cloudcover          0
visibility          0
dtype: int64

In [25]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5702 entries, 0 to 5701
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   datetime          5702 non-null   object 
 1   tempmax           5702 non-null   float64
 2   tempmin           5702 non-null   float64
 3   temp              5702 non-null   float64
 4   dew               5702 non-null   float64
 5   humidity          5702 non-null   float64
 6   precip            5702 non-null   float64
 7   precipprob        5702 non-null   float64
 8   precipcover       5702 non-null   float64
 9   preciptype        5702 non-null   int32  
 10  windgust          5702 non-null   float64
 11  windspeed         5702 non-null   float64
 12  winddir           5702 non-null   float64
 13  sealevelpressure  5702 non-null   float64
 14  cloudcover        5702 non-null   float64
 15  visibility        5702 non-null   float64
dtypes: float64(14), int32(1), object(1)
memory

In [26]:
df_copy['datetime'] = pd.to_datetime(df_copy['datetime'])

df_copy['day'] = df_copy['datetime'].dt.day
df_copy['month'] = df_copy['datetime'].dt.month
df_copy['year'] = df_copy['datetime'].dt.year


C:\Users\Care\AppData\Local\Temp\ipykernel_11792\3815882365.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy['datetime'] = pd.to_datetime(df_copy['datetime'])
C:\Users\Care\AppData\Local\Temp\ipykernel_11792\3815882365.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy['day'] = df_copy['datetime'].dt.day
C:\Users\Care\AppData\Local\Temp\ipykernel_11792\3815882365.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [27]:
df_copy = df_copy.drop(columns=['datetime'])


In [28]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5702 entries, 0 to 5701
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   tempmax           5702 non-null   float64
 1   tempmin           5702 non-null   float64
 2   temp              5702 non-null   float64
 3   dew               5702 non-null   float64
 4   humidity          5702 non-null   float64
 5   precip            5702 non-null   float64
 6   precipprob        5702 non-null   float64
 7   precipcover       5702 non-null   float64
 8   preciptype        5702 non-null   int32  
 9   windgust          5702 non-null   float64
 10  windspeed         5702 non-null   float64
 11  winddir           5702 non-null   float64
 12  sealevelpressure  5702 non-null   float64
 13  cloudcover        5702 non-null   float64
 14  visibility        5702 non-null   float64
 15  day               5702 non-null   int32  
 16  month             5702 non-null   int32  


In [29]:
df_copy.isnull().sum()

tempmax             0
tempmin             0
temp                0
dew                 0
humidity            0
precip              0
precipprob          0
precipcover         0
preciptype          0
windgust            0
windspeed           0
winddir             0
sealevelpressure    0
cloudcover          0
visibility          0
day                 0
month               0
year                0
dtype: int64

In [30]:
df_copy.to_csv('../data/processed/cleaned.csv')